In [1]:
import logging
from pathlib import Path

import pandas as pd
from dotenv import dotenv_values

from generator_drainage_units import run_generator_drainage_units

logging.basicConfig(level=logging.INFO)
pd.set_option("future.no_silent_downcasting", True)

In [2]:
%load_ext autoreload
%autoreload 2

In [13]:
# Define case and base_dir
config = dotenv_values("..\\.env")
base_dir = Path(config["BASE_DIR"])
# case_name
# case_name = "vallei_en_veluwe"
# case_name = "geerestein"
# case_name = "hattemerbroek"
# case_name = "pangelerbeek"
case_name = "Leuvenumse_beek"
case_path = Path(base_dir, case_name)

dir_basisdata = "0_basisdata"
dir_results = "1_resultaat"
ghg_file_name = "GHG_2000-2010_L1.NC"

In [ ]:
gdu = run_generator_drainage_units(
    path=case_path,
    dir_basisdata=dir_basisdata,
    dir_results=dir_results,
    ghg_file_name=ghg_file_name,
    read_results=True,
    write_results=True,
    preprocess=True,
    process=False,
    postprocess=True,
    resolution=2.0,
    depth_waterways=0.2,
    buffer_waterways=2.5,
    smooth_distance=25.0,
    iterations=5,
    iteration_group=100,
)

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 4, figsize=(15, 5))
gdu.drainage_units_0_gdf.plot(ax=axs[0], column="color_id")
gdu.drainage_units_1_gdf.plot(ax=axs[1], column="color_id")
gdu.drainage_units_2_gdf.plot(ax=axs[2], column="color_id")
gdu.drainage_units_3_gdf.plot(ax=axs[3], column="color_id")
titles = ["A/B/C watergangen", "A/B watergangen", "Orde-code", "Stroomgebied"]
for ax, title in zip(axs, titles):
    ax.axis("off")
    ax.set_title(title)
fig.tight_layout()
fig.savefig(Path(gdu.dir_results, "drainage_units.png"), dpi=300)

In [ ]:
gdu.generate_folium_map(
    order_labels=True,
    zmin=7.5,
    zmax=25.0,
    dx=0.0,
    dy=-13.0,
)